# Doc2Vecの動作確認（品詞を落とさず検証）

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) Doc2Vecの動作確認

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、品詞を落とさないようにします。

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 9  # bot_id = 9はセプテーニ
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/17 AM 11:52:10 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/17 AM 11:52:10 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [5]:
_sentences = np.array(questions)
_separated_sentences = Nlang_naive.batch_split(_sentences)

### (2-2) コーパスにタグ付け

models.doc2vecの仕様に従います。

In [6]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [7]:
def doc_to_sentence(sentences, name):
    words = sentences.split(' ')
    return TaggedDocument(words=words, tags=[name])

def corpus_to_sentences(separated_sentences, answer_ids):
    for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
        yield doc_to_sentence(doc, name)

### (2-3) 学習処理／モデルのシリアライズ

In [8]:
sentences = corpus_to_sentences(_separated_sentences, answer_ids)

In [9]:
sentence_list = list(sentences)

In [10]:
model = Doc2Vec(size=500, min_count=1, iter=200)

In [11]:
model.build_vocab(sentence_list)

In [12]:
model.train(sentence_list)

18044965

In [13]:
model_path = 'prototype/better_algorithm/doc2vec.model'

model.save(model_path)

In [14]:
'''
    モデル内に保持されているベクトルの数を取得
    （ラベルを回答IDにすると、ラベルの数が戻る。
    　同一回答IDのサンプルのベクトルが、
    　全て上書きされていると考えられる）
'''
len(model.docvecs)

173

In [15]:
model.docvecs

### (2-4) 予測処理

In [16]:
model_path = 'prototype/better_algorithm/doc2vec.model'

def predict(word, model_path):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

In [17]:
'''
    マウス破損（正解＝4458）
'''
predict('マウス破損', model_path)

(['マウス', '破損'],
 [(4458, 0.6150838136672974),
  (7065, 0.6032382249832153),
  (4598, 0.5798125863075256),
  (4608, 0.5788400173187256),
  (4530, 0.5529136061668396),
  (4578, 0.5507270097732544),
  (4600, 0.542097806930542),
  (4432, 0.538764238357544),
  (7037, 0.5380415320396423),
  (4444, 0.5330274105072021)])

In [18]:
'''
    無線を使用したい（正解＝4516）
'''
predict('無線を使用したい', model_path)

(['無線', 'を', '使用', 'する', 'たい'],
 [(4608, 0.6207529306411743),
  (4434, 0.6170967221260071),
  (4494, 0.6058829426765442),
  (4495, 0.5948004126548767),
  (4516, 0.5931248664855957),
  (4515, 0.5655504465103149),
  (4493, 0.5463182926177979),
  (4610, 0.5420070290565491),
  (4437, 0.5386759042739868),
  (4521, 0.5373212099075317)])

In [19]:
'''
    情報システムのアドレス（正解＝7040）
'''
predict('情報システムのアドレス', model_path)

(['情報', 'システム', 'の', 'アドレス'],
 [(7065, 0.6693086624145508),
  (4588, 0.6203016042709351),
  (4608, 0.6029564142227173),
  (4460, 0.5999393463134766),
  (4420, 0.5916324257850647),
  (4494, 0.5881478786468506),
  (4458, 0.5856335759162903),
  (4437, 0.5844603180885315),
  (4434, 0.5802253484725952),
  (4459, 0.5674356818199158)])

In [20]:
'''
    誤送信防止システムを使いたい（正解＝4432）
'''
predict('誤送信防止システムを使いたい', model_path)

(['誤る', '送信', '防止', 'システム', 'を', '使う', 'たい'],
 [(4533, 0.4564710259437561),
  (4556, 0.4544488787651062),
  (4600, 0.44249314069747925),
  (4557, 0.42799124121665955),
  (7068, 0.4272417426109314),
  (4428, 0.42698192596435547),
  (4596, 0.41918468475341797),
  (7056, 0.41879379749298096),
  (7037, 0.4185285270214081),
  (4498, 0.4164581894874573)])

In [21]:
'''
    携帯からサイボウズを使いたいのですが、どうしたら出来ますか？（正解＝4504）
'''
predict('携帯からサイボウズを使いたいのですが、どうしたら出来ますか？', model_path)

(['携帯',
  'から',
  'サイボウズ',
  'を',
  '使う',
  'たい',
  'の',
  'です',
  'が',
  'どう',
  'する',
  'た',
  '出来る',
  'ます',
  'か'],
 [(7065, 0.5829896926879883),
  (7037, 0.5620793104171753),
  (4472, 0.5277934074401855),
  (7068, 0.5207399129867554),
  (4458, 0.5111536979675293),
  (4509, 0.5046679973602295),
  (4473, 0.4950966238975525),
  (4566, 0.49249356985092163),
  (4468, 0.49069830775260925),
  (7064, 0.490645170211792)])